We will consider from now fixed values c1 and c2 to compare the convergence speed of the contour with respect to the different approaches . For the image “eight.tif”, one can take c1 =110andc2 =227.

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
plt.close('all')
import skimage as ski
from scipy import misc 
from scipy.interpolate import RectBivariateSpline
from PIL import Image
import holoviews as hv
import param
import panel as pn
from panel.pane import LaTeX
hv.extension('bokeh')
import warnings
warnings.filterwarnings('ignore')
import requests
from PIL import Image
from io import BytesIO

In [ ]:
img = np.array( Image.open('coins.png') )[:,:,0] 
options = dict(cmap='gray',xaxis=None,yaxis=None,width=400,height=400,toolbar=None)

In [ ]:
hv.Image(img).opts(**options)

In [ ]:
import plotly.express as px
fig = px.imshow(img, color_continuous_scale='gray')
fig.show()

In [ ]:
img

In [ ]:
"""Divergences of an image""" 
#> pas le bon div ?
def DivHor(x):
    N=len(x[0])
    y=x-np.roll(x,-1,axis=1)
    y[:,0]=-x[:,1]
    y[:,N-1]=x[:,N-1]
    return y

def DivVer(x):
    N=len(x)
    y=x-np.roll(x,-1,axis=0)
    y[0,:]=-x[1,:]
    y[N-1,:]=x[N-1,:]
    return y

def Div(y):
    x=DivHor(y[0])+DivVer(y[1])
    return x

"""Gradients of an image"""

def GradientHor(x):
    y=x-np.roll(x,1,axis=1)
    y[:,0]=0
    return y
def GradientVer(x):
    y=x-np.roll(x,1,axis=0)
    y[0,:]=0
    return y

def Gradient(x):
    return np.array([GradientHor(x),GradientVer(x)])


In [ ]:
def GradientHor(x):
    y=x-np.roll(x,1,axis=1)
    y[:,0]=0
    return y
def GradientVer(x):
    y=x-np.roll(x,1,axis=0)
    y[0,:]=0
    return y
def DivHor(x):
    N=len(x[0])
    y=x-np.roll(x,-1,axis=1)
    y[:,0]=-x[:,1]
    y[:,N-1]=x[:,N-1]
    return y
def DivVer(x):
    N=len(x)
    y=x-np.roll(x,-1,axis=0)
    y[0,:]=-x[1,:]
    y[N-1,:]=x[N-1,:]
    return y
def Gradient(x):
    y=[]
    y.append(GradientHor(x))
    y.append(GradientVer(x))
    return y
def Div(y):
    x=DivHor(y[0])+DivVer(y[1])
    return x

In [ ]:
def PA(u):
    return np.clip(u,0,1)
def normE(grad,E):
    return np.sqrt(grad[0]**2+grad[1]**2+E**2)

In [ ]:
np.shape(img)

In [ ]:
EArray = np.array([1])
muArray = np.array([.5])
lamArray =np.array([0.016,0.017,0.018,0.019,0.02])

In [ ]:
from bokeh.plotting import show

In [ ]:
UK =[]
for mu in muArray:
    for lam in lamArray:
        print(mu,lam)
        #gradient projeté
        tau = 0.1
        E= 1
        I =img
        c1 = 110 
        c2 =227 #à changer
        th=100
        N=200

        u0 = np.ones((658,843))

        uktmp =np.ones((658,843))
        uk = u0

        # for i in range (N):
        
        Delta =th+1
        iterN=0
        while(Delta>th):
            grad = Gradient(uk)
            #gradient projeté
            ukp1 = PA(uk + tau*(Div(grad/(normE(grad,E))) - lam*(I-c1)**2 - lam*(I-c2)**2))
            #Affichage u1
            if (iterN==0):
                fig = px.imshow(ukp1, color_continuous_scale='gray')
                fig.show()
            #Calcul pour CV
            Delta =np.linalg.norm(ukp1-uk)
            uk=ukp1
            iterN+=1
        #Affichage uK qui a CV
        fig = px.imshow(uk, color_continuous_scale='gray')
        fig.show()
        #Affichage 
        uk[uk<mu]=0
        uk[uk>mu]=255
        fig = px.imshow(uk, color_continuous_scale='gray')
        fig.show()
        UK += [uk]

In [ ]:
uk[uk<mu]=0
uk[uk>mu]=255

In [ ]:
uk

In [ ]:
Itmp = I/255
Itmp[Itmp<mu]=0
Itmp[Itmp>mu]=255



In [ ]:
hv.Image(Itmp).opts(**options)

Implement algorithm (3.7) and test it with the time step τ ≈ 0.1. The threshold μ = 0.5 can be considered to define the binary region of interest: ub(x) = 1 if u(x) > 0.5 and 0 otherwise.

Is the result sensible to the initialization of u0?

Implement a function compute_energy_smooth that takes a binary map u, (I − c1)2, (I − c2 )2 , λ and ε as arguments and returns the value of functional (3.6). As c1 and c2 are now fixed, the matrix (I − c1)2 and (I − c2)2 can be precomputed to save computational time. Look at the evolution of the functional value along iterations k for the Chan-Vese model and the Chan-Esedoglu-Nikolova one. For the Chan and Vese model, one should take as input argument u = H(φ).